# Parse Ensembl Gene Annotation Gff3 file and Generate json Files

This notebook parses the Ensembl gene annotation GFF3 file, and generates a corresponding json file per gene.

The GFF3 file downloaded from [Ensembl gene anotation](https://useast.ensembl.org/Homo_sapiens/Info/Index), by selecting to download GFF3.

See [GFF3 format](https://www.ensembl.org/info/website/upload/gff3.html). See also [here](http://ftp.ensembl.org/pub/release-108/gff3/homo_sapiens/README).

Note: In general, a protein coding gene may contain non-coding transcripts (RNAs), e.g. lnc_RNA.

**To generate the json annotation files:**
1. Set the GFF3 file name (`gff3_file` in the `Load the GFF3 File` section below).
2. Set the json path (`json_path` in the `Parse GFF3 File and Generate json Files` section below).
3. Set `gene_types_to_include` list (in both cells below), which defines the Type of genes to parse. Should be `['gene', 'ncRNA_gene', 'pseudogene']`, or any non-empty subset of it. `'gene'` implies all protein coding genes.
4. Run these two sections.

**To update to a new Ensembl version follow these steps:**
1. Download the GFF3 file (e.g. `Homo_sapiens.GRCh38.109.gff3.gz`, from `https://ftp.ensembl.org/pub/release-109/gff3/homo_sapiens/`) to the path defined by the `gff3_file` variable below (and update `gff3_file` to the new file name).
2. Update the `json_path` variable below to the json denstination folder.
3. Run this notebook.
4. In `Utils/gene_anot_utils.py` update the function `gene2json_file` with the new json denstination folder.

**To generate a json file of a gene (or to generate the annotation dictionary of a gene that is written to the json file):**
1. Run the `Load the GFF3 File` section below.
2. Set the gene name in the `genes` variable in the `Parse GFF3 File and Generate json Files` section below (e.g. `genes = ['EGFR']`).
3. Set the other parameters in the `Parse GFF3 File and Generate json Files` section below (e.g., if json file is not needed, set `generate_json_files = False`).
4. Run the sections. The variable `gene_anot_s` will then hold the gene annotation dictionary.


In [1]:
'''
Global configuration (common to all projects).
'''
import pandas as pd
import numpy as np
import os, sys, re
import pathlib
import pickle
import matplotlib.pyplot as plt
import glob
# import subprocess

# make sure PYTHONPATH is set, containing the Utils folder.
import mysys_init as si

# if sys.platform == 'darwin':
#     MACHINE = 'MAC'
#     ROOT_path = '/Users/yoramzarai/work/mystuff/Ramot/Projects/Cancer_mut/'
#     chromosome_path = pathlib.Path(ROOT_path) / 'Explore_Data' / 'GRCh38_chr'
# else:  # Linux TAU machines
#     MACHINE = 'POWER'
#     ROOT_path = '/tamir2/yoramzar/Projects/Cancer_mut/'
#     chromosome_path = pathlib.Path('/tamir2/lab_resources/Genomes/Human/human_hg38/Chromosome')
#     #chromosome_path = pathlib.Path(ROOT_path) / 'cancer_proj' / 'human_hg38' / 'Chromosome'
    
# # adding Utils/ folder to path
# utils_path = pathlib.Path(ROOT_path) / 'Utils' 
# sys.path.append(str(utils_path))

In [2]:
import json
import ensembl_gene_annotations_utils as egna

# Load the GFF3 File

The main gene types in the 'Type' column are:
1. gene - protein coding genes.
2. ncRNA_gene - non-protein coding genes.
3. pseudogene

See [this](https://useast.ensembl.org/info/genome/genebuild/biotypes.html) for more information.

Most of the other values in the 'Type' column are sub-types of these (for example, exon, CDS, lnc_RNA, etc.)

For some of the genes, there is no "Name=\<name\>" string in the Attributes column. This string is available, however, in all protein coding genes. 

In [3]:
gff3_file  = pathlib.Path(si.ROOT_path) / 'Data' / 'Ensembl_gene_annotation' / 'Homo_sapiens.GRCh38.109.gff3'
# ============================================================================================================

df_anot = pd.read_csv(gff3_file, sep='\t', comment='#', low_memory=False,
                      names=['Chrm', 'Source', 'Type', 'Start', 'End', 'Score', 'Strand', 'Phase', 'Attributes'],
                      dtype={
                        'Chrm': str,
                         'Source': str,
                         'Type': str,
                         'Start': int,  #  1-based
                         'End': int,    #  1-based.
                         'Score': str,
                         'Strand': str,  # '+' for positive strand, '-' for negative strand.
                         'Phase': str,  # some of the rows have non-integer type
                         'Attributes': str
                     })
display(df_anot)

# all different types
all_types = df_anot['Type'].unique().tolist()
print(f"Type values:\n{all_types}")
df_anot.info()

,Chrm,Source,Type,Start,End,Score,Strand,Phase,Attributes
0,1,GRCh38,chromosome,1,248956422,.,.,.,"ID=chromosome:1;Alias=CM000663.2,chr1,NC_00000..."
1,1,.,biological_region,10469,11240,1.3e+03,.,.,external_name=oe %3D 0.79;logic_name=cpg
2,1,.,biological_region,10650,10657,0.999,+,.,logic_name=eponine
3,1,.,biological_region,10655,10657,0.999,-,.,logic_name=eponine
4,1,.,biological_region,10678,10687,0.999,+,.,logic_name=eponine
...,...,...,...,...,...,...,...,...,...
3414006,Y,.,biological_region,26626966,26627137,0.994,-,.,external_name=rank %3D 1;logic_name=firstef
3414007,Y,.,biological_region,26627457,26628186,0.997,+,.,external_name=rank %3D 1;logic_name=firstef
3414008,Y,havana,pseudogene,56855244,56855488,.,+,.,ID=gene:ENSG00000235857;Name=CTBP2P1;biotype=p...
3414009,Y,havana,pseudogenic_transcript,56855244,56855488,.,+,.,ID=transcript:ENST00000431853;Parent=gene:ENSG...


Type values:
['chromosome', 'biological_region', 'ncRNA_gene', 'lnc_RNA', 'exon', 'pseudogene', 'pseudogenic_transcript', 'miRNA', 'gene', 'mRNA', 'five_prime_UTR', 'CDS', 'three_prime_UTR', 'snRNA', 'transcript', 'ncRNA', 'unconfirmed_transcript', 'snoRNA', 'scRNA', 'rRNA', 'V_gene_segment', 'D_gene_segment', 'J_gene_segment', 'C_gene_segment', 'scaffold', 'tRNA']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3414011 entries, 0 to 3414010
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Chrm        object
 1   Source      object
 2   Type        object
 3   Start       int64 
 4   End         int64 
 5   Score       object
 6   Strand      object
 7   Phase       object
 8   Attributes  object
dtypes: int64(2), object(7)
memory usage: 234.4+ MB


In [4]:
"""
Get all required gene names (based on gene_types_to_include variable).
"""
# set the gene Type to include. Must be ['gene', 'ncRNA_gene', 'pseudogene'] or any non-empty subset of it
gene_types_to_include = ['gene']
# =======================================================================================================

def get_gene_name(x: pd.Series) -> str:
    """
    If the Attributes has "Name=<name>", we return <name>. Otherwise we return <gene_id> 
    (from the "gene_id=<gene_id" string in the Attributes column).
    """
    try:
        # using the gene name
        return dict([tuple(y.split('=')) for y in x['Attributes'].split(';')])['Name']
    except:
        # noval transcript (does not have a Name field). Using the gene ID instead. 
        return dict([tuple(y.split('=')) for y in x['Attributes'].split(';')])['gene_id']
# ----------------------------------------------------------------------------------------
gene_types = ','.join(gene_types_to_include)
print(f"Selecting {gene_types} Types...")

qry = ''.join([f"Type == '{gene_types_to_include[0]}'"] + [f" or Type == '{g}'" for g in gene_types_to_include[1:]])
#print(f"""{qry=}""")
df_genes = df_anot.query(qry)
    
# add gene name column
df_genes.insert(1, 'Gene_name', df_genes.apply(get_gene_name, axis=1))
print(f"\nDataframe containing {gene_types} Type:")
display(df_genes)

all_genes = df_genes['Gene_name'].unique().tolist()
print(f"There are {len(all_genes):,} {gene_types} genes.")

Selecting gene Types...

Dataframe containing gene Type:


,Chrm,Gene_name,Source,Type,Start,End,Score,Strand,Phase,Attributes
84,1,OR4F5,ensembl_havana,gene,65419,71585,.,+,.,ID=gene:ENSG00000186092;Name=OR4F5;biotype=pro...
370,1,OR4F29,ensembl_havana,gene,450740,451678,.,-,.,ID=gene:ENSG00000284733;Name=OR4F29;biotype=pr...
502,1,OR4F16,ensembl_havana,gene,685716,686654,.,-,.,ID=gene:ENSG00000284662;Name=OR4F16;biotype=pr...
1097,1,SAMD11,ensembl_havana,gene,923923,944575,.,+,.,ID=gene:ENSG00000187634;Name=SAMD11;biotype=pr...
1459,1,NOC2L,ensembl_havana,gene,944203,959309,.,-,.,ID=gene:ENSG00000188976;Name=NOC2L;biotype=pro...
...,...,...,...,...,...,...,...,...,...,...
3412977,Y,BPY2B,ensembl_havana,gene,24607560,24639207,.,+,.,ID=gene:ENSG00000183795;Name=BPY2B;biotype=pro...
3413043,Y,DAZ3,ensembl_havana,gene,24763069,24813492,.,-,.,ID=gene:ENSG00000187191;Name=DAZ3;biotype=prot...
3413169,Y,DAZ4,ensembl_havana,gene,24833843,24907040,.,+,.,ID=gene:ENSG00000205916;Name=DAZ4;biotype=prot...
3413567,Y,BPY2C,ensembl_havana,gene,25030901,25062548,.,-,.,ID=gene:ENSG00000185894;Name=BPY2C;biotype=pro...


There are 21,519 gene genes.


# Parse GFF3 File and Generate json Files

Here we parse the `df_anot` dataframe and generate the json files.

In [5]:
%%time
"""
Parse genes
"""
# set a list of gene names to generate json files for. In general, some of the genes do not have 
# name associated with them, and in this case the gene ID is used as the name. See the cell above.
genes = all_genes
#genes = ['EGFR', 'IDH1', 'PIK3CA']

# this should be identical to the value set above
#gene_types_to_include = ['gene']

#generate_json_files = False
generate_json_files = True

# folder to save all json files
json_path = pathlib.Path(si.ROOT_path) / 'Ensembl_gene_anot' / 'Ensembl_109_gene_annotations' if si.MACHINE=='MAC' else \
pathlib.Path('/tamir2/lab_resources/Genomes/Human/human_hg38/Ensembl_109_gene_annotations')
# =============================================================================================================================================================================================

# All Type values for a gene. This is for verification only.
gene_type_values = ['gene', 'ncRNA_gene', 'pseudogene']

# gene's transcript types to process
types_transcript_processed = ['mRNA', 'lnc_RNA', 'pseudogenic_transcript', 
                              'ncRNA', 'miRNA', 'snoRNA',
                              'snRNA', 'scRNA', 'rRNA', 'tRNA']

# sub-transcript types to process
types_in_transcript = ['CDS', 'exon', 'five_prime_UTR', 'three_prime_UTR']
# --------------------------------------------------------------------------

print(f"Parsing GFF3 for {len(genes):,} genes of Type {gene_types_to_include} (generating json file = {generate_json_files}) ...")

if generate_json_files:
    json_path.mkdir(exist_ok=True)

for i, gene in enumerate(genes, start=1):
    if i % 5000 == 0:
        print(f"Done {i:,} genes.")

    # generate a dataframe starting from the gene of interest (the dataframe may contain other genes as well, which will be ignored)
    if (df_gene_all := egna.get_df_start_with_gene(df_anot, gene, gene_types_to_include)) is None:
        continue
        
    # verify
    first_row = df_gene_all.iloc[0]
    if first_row['Type'] not in gene_type_values:
        print(f"df_gene_all for {gene=} does not start with a row with a Type in {gene_type_values} !!")
        continue
        
    d = dict([tuple(x.split('=')) for x in first_row['Attributes'].split(';')])
    try:
        first_row_gene = d['Name']
    except KeyError:
        # non-coding genes have no "Name" field. In these cases we use the gene ID as the name.
        first_row_gene = d['gene_id']
        
    if gene != first_row_gene:
        print(f"{first_row_gene=} differ from {gene=} !!")
        continue
    
    # generate the annotation dictionary
    if (gene_anot_s := egna.parse_ensembl_gene_anot_gene(df_gene_all, types_transcript_processed, types_in_transcript)) == {}:
        print(f"Error in annotating {gene=} !!")
    else:
        if generate_json_files:
            with open(json_path / f"{gene}_annotation.json", 'w') as fp:
                json.dump(gene_anot_s, fp)
    
print(f"Done parsing {len(genes):,} genes.")

Parsing GFF3 for 21,519 genes of Type ['gene'] (generating json file = True) ...
Done 5,000 genes.
Done 10,000 genes.
Done 15,000 genes.
Done 20,000 genes.
Done parsing 21,519 genes.
CPU times: user 1h 34min 30s, sys: 10min 27s, total: 1h 44min 57s
Wall time: 1h 45min


In [ ]:
# display annotation for the last gene
display(df_gene_all)
egna.display_gene_annotation_structure(gene_anot_s)

# ScratchPad
Ignore this section.

## Testing

In [ ]:
import mRNA_utils as mrnau
import ensembl_rest_utils as erst
import translation as tran

get_chrm_file_name = mrnau.get_chrm_file_name_mac if MACHINE=='MAC' else mrnau.get_chrm_file_name_power
mrna_data_path = pathlib.Path(ROOT_path) / 'Explore_Splice' / 'mrna_data' if MACHINE=='MAC' else pathlib.Path('/tamir2/lab_resources/Genomes/Human/human_hg38/mRNAsAll_June2021/')

report_path = pathlib.Path(ROOT_path) / 'Ensembl_gene_anot' / 'Reports'

report_path.mkdir(exist_ok=True)

In [ ]:
compare_with_REST = True
indx = 1 #1 # pick transcript index to test
# ========================================

def get_sequence_from_start_end_segments(start: list, end: list, rev: bool, chrm_path: str, offset: int=0) -> str:
    """
    Get the chromosome sequence, at the correponding strand (based on rev input) from 
    the concatenation of segments defined by start and end list.
    
    rev == True implies start[i]>end[i], start[j]<start[i], end[j]<end[i], for j>i, otherwise 
                        start[i]<end[i], start[j]>start[i], end[j]>end[i], for j>1. 
    """
    # convert to increasing lists
    a_start, a_end = (end[::-1], start[::-1]) if rev else (start, end)
    seq = ''.join([mrnau.pull_fasta_seq(chrm_path, a_s, 0, a_e - a_s, False)
                   for a_s, a_e in zip(a_start, a_end)])
    return tran.reverse_complement(seq)[offset:] if rev else seq[offset:]
# ========================================================================================


test_transcript = gene_anot_s['transcripts'][indx]
print(f"Transcript=\n{test_transcript}")

if test_transcript['transcript_biotype'] != 'protein_coding':
    raise Exception(f"Exception: transcript{indx} ({test_transcript['transcript_name']}) corresponds to a non protein coding transcript !!")


chrm = f"chr{gene_anot_s['chrm']}"
rev = gene_anot_s['rev']
transcript_id = test_transcript['transcript_id']
transcript_name = test_transcript['transcript_name']
p_id = test_transcript['protein_id']

cds_offset = test_transcript['CDS_phase'][0]

print(f"\n{gene=}, {transcript_id=}, {p_id=}, {transcript_name=}, {chrm=}, {rev=}")

if compare_with_REST:
    ensb_trans = erst.lookup_id(transcript_id)
    ensb_pid = ensb_trans['Translation']['id']
    print(f"\tREST: {transcript_id=} ==> {ensb_pid=}")
    assert ensb_pid == p_id, f"{p_id=} and {ensb_pid=} mismatch !!"
    ensb_aa = erst.sequence_endpoint_base(ensb_pid)
    
    # need to check this more carefully
    if cds_offset != 0:
        ensb_aa = ensb_aa[1:]


# comparing with Transcripts in mRNA_utils
trans_obj = mrnau.Transcripts(gene)
chrm_path = chromosome_path / get_chrm_file_name(trans_obj.chrm)
aa = trans_obj.AA(transcript_id, chrm_path)
if aa is not None:
    aa_n = aa[:-1] if aa[-1] == '?' else aa


    # compare with REST
    if compare_with_REST:
        if ensb_aa == aa_n:
            print(f"\tAA matches REST AA.")
        else:
            print(f"AA and REST do not agree !!")
    else:
        print(f"Comparing with REST is set to OFF !")
    
"""
Get AA from gene_anot_s
"""
ORF = get_sequence_from_start_end_segments(test_transcript['CDS_start'], test_transcript['CDS_end'], rev, chrm_path, offset=cds_offset)
# this includes the stop codon
aa_s = tran.convert_aa_32IUPAC(tran.translate(ORF))

aa_s_n = aa_s[:-1] if aa_s[-1] == '?' else aa_s

if aa is not None:
    if aa_s_n == aa_n:
        print(f"\nAA matches with gene_anot_s")
    else:
        print(f"\nMissmatch between gene_anot_s and mrnau !!!!!")
    
if compare_with_REST:
    if aa_s_n == ensb_aa:
        print(f"\nREST matches with gene_anot_s")
    else:
        print(f"\nMissmatch between gene_anot_s and REST !!!!!")
    
"""
Get mRNA (transcript) sequence
"""
mRNA_seq = get_sequence_from_start_end_segments(test_transcript['exon_start'], test_transcript['exon_end'], rev, chrm_path)

"""
Get 5'UTR
"""
if test_transcript['5UTR_start'] == []:
    print("No 5UTR")
else:
    UTR5_seq = get_sequence_from_start_end_segments(test_transcript['5UTR_start'], test_transcript['5UTR_end'], rev, chrm_path)
    print(f"\n5UTR=\n{UTR5_seq}")

"""
Get 3UTR
"""
if test_transcript['3UTR_start'] == []:
    print("No 3UTR")
else:
    UTR3_seq = get_sequence_from_start_end_segments(test_transcript['3UTR_start'], test_transcript['3UTR_end'], rev, chrm_path)
    print(f"\n3UTR=\n{UTR3_seq}")

In [ ]:
print(aa_s_n)

In [ ]:
print(ensb_aa)

In [ ]:
print(ORF[:10])